Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.


In [22]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from math import ceil

import numpy as np

In [2]:
PROBLEM_SIZE = 5
NUM_SETS = 10
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)
)
State = namedtuple("State", ["taken", "not_taken"])

In [3]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

In [8]:
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

In [4]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

In [46]:
def g(state):
    return len(state.taken)


def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(
        sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS
    )
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted(
        (sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS),
        reverse=True,
    )
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


# My Heuristics
def h4(state):
    already_covered = covered(state)
    remaining_sets = len(state.not_taken)
    missing_elements = PROBLEM_SIZE - sum(already_covered)

    # Estimate based on the ratio of remaining sets to missing elements
    return missing_elements / remaining_sets


# Estimates the number of sets that can directly cover the remaining uncovered elements.
# It counts how many sets have **at least one element in common with the uncovered elements**.
# It's an estimate of how many sets might be needed to cover the remaining elements efficiently.
# This solution was done by the help of GPT-3.5 model
def h5(state):
    # Check if all elements are already covered in the current state.
    already_covered = covered(state)
    if np.all(already_covered):
        return 0  # If all elements are covered, no additional steps are needed.

    # Identify elements that are not yet covered.
    uncovered_elements = np.logical_not(already_covered)

    # Create an array of candidates, where each element represents
    # whether a set can directly cover the remaining uncovered elements.
    candidates = np.array([any(np.logical_and(s, uncovered_elements)) for s in SETS])

    # Estimate the number of sets that can directly cover the remaining elements.
    to_cover = np.sum(candidates)

    return to_cover


def f(state):
    return g(state) + h5(state)  # Combine g and h into f

In [47]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

Solved in 4 steps (3 tiles)


In [12]:
current_state

State(taken={9, 3, 7}, not_taken={0, 1, 2, 4, 5, 6, 8})

In [13]:
goal_check(current_state)

True